In [5]:
from xml.etree import ElementTree
from xml.dom import minidom
from xml.etree.ElementTree import Element, SubElement, Comment, indent

def prettify(elem):
    """Return a pretty-printed XML string for the Element.
    """
    rough_string = ElementTree.tostring(elem, encoding="ISO-8859-1")
    reparsed = minidom.parseString(rough_string)
    return reparsed.toprettyxml(indent="\t")

In [6]:
import numpy as np
import os

valve_start = 1

hyb_start = 1

reg_start = 2

num_rounds = 16

add_ref_hyb = True

#data_type = 'R'
data_type = 'C'

valve_ids = np.arange(valve_start, valve_start + num_rounds)
hyb_ids = np.arange(hyb_start, hyb_start + num_rounds)
reg_names = [f'{data_type}{_i}' for _i in np.arange(reg_start, reg_start + num_rounds)]

In [7]:
source_folder = r'D:\Cosmos_iN\20250517_IntronMERFISH_WT_rep1'
#target_drive = r'\\10.245.74.158\Chromatin_NAS_8'
target_drive = r'\\10.245.74.212\Chromatin_NAS_9'

#imaging_protocol = r'Zscan_750_647_488_s13_n1000_8Hz'
imaging_protocol = r'Zscan_750_647_488_s50_n250_10Hz'

#imaging_protocol = r'Conv_Zscan_750_s13_647_s13_560_s1_405_s1_n1000_5Hz'

#ref_imaging_protocol = r'Zscan_750_647_488_405_s13_n1000_8Hz'
#ref_imaging_protocol =r'Conv_Zscan_750_647_488_405_s50_n250_5Hz'
ref_imaging_protocol = r'Zscan_750_647_488_405_s50_n250_10Hz'

ref_hyb = 0

In [8]:
cmd_seq = Element('command_sequence')

if add_ref_hyb:
    # add hyb 0
    # comments
    comment = Comment(f"Hyb 0")
    cmd_seq.append(comment)
    # flow imaging buffer
    imaging = SubElement(cmd_seq, 'valve_protocol')
    imaging.text = f"Flow Image Buffer"

    # change directory
    change_dir = SubElement(cmd_seq, 'change_directory')
    change_dir.text = os.path.join(source_folder, f"H0{data_type}1")
    # wakeup
    wakeup = SubElement(cmd_seq, 'wakeup')
    wakeup.text = "15000"

    # imaging loop
    _im_p = ref_imaging_protocol

    loop = SubElement(cmd_seq, 'loop', name='Position Loop Zscan', increment="name")
    loop_item = SubElement(loop, 'item', name=_im_p)
    loop_item.text = " "
    # delay time
    delay = SubElement(cmd_seq, 'delay')
    delay.text = "2000"
    # copy folder
    copy_dir = SubElement(cmd_seq, 'copy_directory')
    source_dir = SubElement(copy_dir, 'source_path')
    source_dir.text = change_dir.text 
    target_dir = SubElement(copy_dir, 'target_path')
    target_dir.text = os.path.join(target_drive, 
                                   os.path.basename(os.path.dirname(source_dir.text)), 
                                   os.path.basename(source_dir.text))
    del_source = SubElement(copy_dir, 'delete_source')
    del_source.text = "True"


for _i, (_vid, _hid, _rname) in enumerate(zip(valve_ids, hyb_ids, reg_names)):
    
    # select protocol
    if _hid == ref_hyb:
        _im_p = ref_imaging_protocol
    else:
        _im_p = imaging_protocol
    
    if _hid > 0:
        # TCEP
        tcep = SubElement(cmd_seq, 'valve_protocol')
        tcep.text = "Cleave"
        
    # comments
    comment = Comment(f"Hyb {_hid} with {_vid} for {_rname}")
    cmd_seq.append(comment)
    
    # Adaptor
    adt = SubElement(cmd_seq, 'valve_protocol')
    adt.text = f"Hybridize {_vid}"
    
    # Readouts
    readouts = SubElement(cmd_seq, 'valve_protocol')
    readouts.text = "Flow Readout"
    
    # Imaging buffer
    imaging = SubElement(cmd_seq, 'valve_protocol')
    imaging.text = f"Flow Image Buffer"
    # change directory
    change_dir = SubElement(cmd_seq, 'change_directory')
    change_dir.text = os.path.join(source_folder, f"H{_hid}{_rname.upper()}")
    # wakeup
    wakeup = SubElement(cmd_seq, 'wakeup')
    wakeup.text = "15000"
    
    # imaging loop
    loop = SubElement(cmd_seq, 'loop', name='Position Loop Zscan', increment="name")
    loop_item = SubElement(loop, 'item', name=_im_p)
    loop_item.text = " "
    # delay time
    delay = SubElement(cmd_seq, 'delay')
    delay.text = "2000"
    # copy folder
    copy_dir = SubElement(cmd_seq, 'copy_directory')
    source_dir = SubElement(copy_dir, 'source_path')
    source_dir.text = change_dir.text#cmd_seq.findall('change_directory')[-1].text
    target_dir = SubElement(copy_dir, 'target_path')
    target_dir.text = os.path.join(target_drive, 
                                   os.path.basename(os.path.dirname(source_dir.text)), 
                                   os.path.basename(source_dir.text))
    del_source = SubElement(copy_dir, 'delete_source')
    del_source.text = "True"
    # empty line
    indent(target_dir)

final_wash = SubElement(cmd_seq, 'valve_protocol')
final_wash.text = f"Flow Wash Buffer"

final_str = prettify(cmd_seq)
    
print( final_str )

<?xml version="1.0" ?>
<command_sequence>
	<!--Hyb 0-->
	<valve_protocol>Flow Image Buffer</valve_protocol>
	<change_directory>D:\Cosmos_iN\20250517_IntronMERFISH_WT_rep1\H0C1</change_directory>
	<wakeup>15000</wakeup>
	<loop name="Position Loop Zscan" increment="name">
		<item name="Zscan_750_647_488_405_s50_n250_10Hz"> </item>
	</loop>
	<delay>2000</delay>
	<copy_directory>
		<source_path>D:\Cosmos_iN\20250517_IntronMERFISH_WT_rep1\H0C1</source_path>
		<target_path>\\10.245.74.212\Chromatin_NAS_9\20250517_IntronMERFISH_WT_rep1\H0C1</target_path>
		<delete_source>True</delete_source>
	</copy_directory>
	<valve_protocol>Cleave</valve_protocol>
	<!--Hyb 1 with 1 for C2-->
	<valve_protocol>Hybridize 1</valve_protocol>
	<valve_protocol>Flow Readout</valve_protocol>
	<valve_protocol>Flow Image Buffer</valve_protocol>
	<change_directory>D:\Cosmos_iN\20250517_IntronMERFISH_WT_rep1\H1C2</change_directory>
	<wakeup>15000</wakeup>
	<loop name="Position Loop Zscan" increment="name">
		<item name

In [9]:
#os.listdir(r'\\10.245.74.158\Chromatin_NAS_4\20220402-P_brain_CTP11-13_from_0329')

In [10]:
if not os.path.exists(source_folder):
    os.makedirs(source_folder)
    
save_filename = os.path.join(source_folder, f"generated_dave_H{hyb_start}-{hyb_start+num_rounds-1}.txt")
with open(save_filename, 'w') as _output_handle:
    print(save_filename)
    _output_handle.write(final_str)

D:\Cosmos_iN\20250517_IntronMERFISH_WT_rep1\generated_dave_H1-16.txt


In [12]:
<event>
    <channel>0</channel>
    <power>1</power>
    <on>0.0</on>
    <off>1.0</off>
    <color>255,255,255</color>
  </event>
  <event>
    <channel>1</channel>
    <power>1</power>
    <on>1.0</on>
    <off>2.0</off>
    <color>255,255,255</color>
  </event>
  <event>
    <channel>3</channel>
    <power>1</power>
    <on>2.0</on>
    <off>3.0</off>
    <color>255,255,255</color>
  </event>

0.0,0.0,0.0,0.25,0.25,0.25,0.5,0.5,0.5,0.75,0.75,0.75,1.0,1.0,1.0,1.25,1.25,1.25,1.5,1.5,1.5,1.75,1.75,1.75,2.0,2.0,2.0,2.25,2.25,2.25,2.5,2.5,2.5,2.75,2.75,2.75,3.0,3.0,3.0,3.25,3.25,3.25,3.5,3.5,3.5,3.75,3.75,3.75,4.0,4.0,4.0,4.25,4.25,4.25,4.5,4.5,4.5,4.75,4.75,4.75,5.0,5.0,5.0,5.25,5.25,5.25,5.5,5.5,5.5,5.75,5.75,5.75,6.0,6.0,6.0,6.25,6.25,6.25,6.5,6.5,6.5,6.75,6.75,6.75,7.0,7.0,7.0,7.25,7.25,7.25,7.5,7.5,7.5,7.75,7.75,7.75,8.0,8.0,8.0,8.25,8.25,8.25,8.5,8.5,8.5,8.75,8.75,8.75,9.0,9.0,9.0,9.25,9.25,9.25,9.5,9.5,9.5,9.75,9.75,9.75,

In [ ]:
print()